In [107]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [108]:
clf = pd.read_csv("Data_for_practice/client_test.csv")

In [109]:
clf['creation_date'] = pd.to_datetime(clf['creation_date'])

In [110]:
clf['coop_time'] = (2021 - clf['creation_date'].dt.year)*12 - clf['creation_date'].dt.month

In [111]:
df = pd.read_csv("Data_for_practice/test_invoice.csv") 

In [112]:
df['counter_type'] = df['counter_type'].map({"ELEC":1,"GAZ":0})

In [113]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'], dayfirst=True)
df['invoice_month'] = df['invoice_date'].dt.month

In [114]:
df['delta_index'] = df['new_index'] - df['old_index']

In [115]:
df['delta_time'] = df.sort_values(['client_id','invoice_date']).groupby('client_id')['invoice_date'].diff(periods = 1).dt.days.reset_index(drop=True)

In [116]:
df["delta_time"] = df["delta_time"].fillna(0)

In [117]:
df = df.drop(columns = ["invoice_date","counter_number","counter_code","consommation_level_4","old_index","new_index"])

In [118]:
 l  = ['tarif_type', 'counter_statue', 'reading_remarque','counter_coefficient', 'consommation_level_1', 'consommation_level_2','consommation_level_3', 'months_number', 'counter_type','invoice_month', 'delta_index', 'delta_time',]

In [119]:
new_df = df.groupby('client_id')[l].agg(['mean','std'])
new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
new_df.reset_index(inplace=True)

In [120]:
dff = df.groupby('client_id').size().reset_index(name='transactions_count')

In [121]:
full_df = pd.merge(clf, new_df, on='client_id', how='left')

In [122]:
full_df = pd.merge(full_df, dff, on='client_id', how='left')

In [123]:
full_df['invoice_per_cooperation'] = full_df['transactions_count'] / full_df['coop_time']

In [124]:
full_df = full_df.fillna(0)

In [125]:
full_df = full_df.drop(columns = ["creation_date"])

In [127]:
full_df

,disrict,client_id,client_catg,region,coop_time,tarif_type_mean,tarif_type_std,counter_statue_mean,counter_statue_std,reading_remarque_mean,...,counter_type_mean,counter_type_std,invoice_month_mean,invoice_month_std,delta_index_mean,delta_index_std,delta_time_mean,delta_time_std,transactions_count,invoice_per_cooperation
0,62,train_Client_675,11,301,44,11.000000,0.000000,0.000000,0.000000,9.000000,...,1.000000,0.000000,7.428571,3.457222,452.285714,303.116982,104.857143,67.040855,7,0.159091
1,63,train_Client_6750,11,313,138,21.489362,14.084948,0.021277,0.145865,7.319149,...,0.638298,0.485688,4.957447,2.553407,181.638298,146.618171,76.255319,91.155259,47,0.340580
2,62,train_Client_67500,11,301,456,25.000000,15.111526,0.000000,0.000000,6.720588,...,0.500000,0.503718,8.264706,3.432147,391.250000,288.477960,71.632353,90.812239,68,0.149123
3,60,train_Client_67501,11,101,140,11.000000,0.000000,0.058824,0.242536,8.058824,...,1.000000,0.000000,4.882353,3.497899,924.176471,868.996493,207.823529,147.635207,17,0.121429
4,62,train_Client_67502,11,310,336,17.645833,12.317587,0.000000,0.000000,7.687500,...,0.770833,0.424744,4.270833,2.811372,1731.229167,1689.180630,102.583333,108.754816,48,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36105,62,train_Client_99995,11,304,197,24.887324,14.590162,0.070423,0.593391,6.000000,...,0.521127,0.503109,6.042254,3.146139,0.000000,0.000000,71.816901,72.197806,71,0.360406
36106,63,train_Client_99996,11,311,98,25.853659,14.675764,0.268293,0.837389,8.487805,...,0.487805,0.506061,6.048780,3.470960,186.609756,203.347840,56.268293,68.577338,41,0.418367
36107,63,train_Client_99997,11,311,109,22.277778,14.337984,0.000000,0.000000,9.000000,...,0.611111,0.494413,6.888889,3.732313,273.083333,211.123441,78.361111,100.941894,36,0.330275
36108,60,train_Client_99998,11,101,324,10.000000,0.000000,0.000000,0.000000,7.500000,...,1.000000,0.000000,10.000000,2.828427,370.500000,232.638131,61.000000,86.267027,2,0.006173


In [128]:
full_df.to_csv("first_reconizance_part1_test.csv")